# Pop2Piano - Colab All-in-One

This notebook provides a complete environment to run Pop2Piano on Google Colab, including fixes for dependencies and legacy code issues.

## Features:
1. **Environment Setup:** Installs all necessary libraries and system packages.
2. **Dependency Fixes:** Implements a workaround for the 'essentia' library issue that causes `transformers` to crash.
3. **Data Download:** Runs the updated `download.py` script to fetch the training dataset (optional).
4. **Inference (Pop2Piano):** Allows you to upload an MP3/WAV file and generate a Piano Cover MIDI + Audio file.

In [ ]:
# @title 1. Install Dependencies 📦
import os
import sys
import subprocess

print("Installing dependencies... ⏳")

# Install Python packages
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/huggingface/transformers.git"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "librosa", "pretty_midi", "midi2audio", "scipy", "yt-dlp", "imageio-ffmpeg", "omegaconf", "joblib", "tqdm"])

# Install System packages (FluidSynth for MIDI to Audio conversion)
os.system("sudo apt-get update -q")
os.system("sudo apt-get install -y -q fluidsynth fluid-soundfont-gm ffmpeg")

print("Done! ✅")

In [ ]:
# @title 2. Apply Magic Fixes (Essentia Mocking) 🪄
import sys
from unittest.mock import MagicMock

# This "tricks" the system into believing 'essentia' is installed.
# This effectively bypasses the ImportError that stops 'transformers' from loading the implementation.
sys.modules["essentia"] = MagicMock()
sys.modules["essentia.standard"] = MagicMock()

print("Essentia mocked successfully! Transformers will work now. 🧙‍♂️")

In [ ]:
# @title 3. Run Download Script (Optional) 📥
# Runs download.py to fetch the training dataset.
# IMPORTANT: Ensure you have uploaded 'download/download.py' and 'train_dataset.csv' to Colab.

# If you are cloning the repo un-comment the lines below:
# !git clone https://github.com/sweetcocoa/pop2piano.git
# %cd pop2piano

# Otherwise, we assume files are in the current directory or you adjust paths.
# Change max_size_gb to a number (e.g., 5.0) to limit size, or remove it for unlimited.

!python download/download.py train_dataset.csv output_dir/ --max_size_gb 10.0

# To run without limit:
# !python download/download.py train_dataset.csv output_dir/

In [ ]:
# @title 4. Run Inference: Pop Song -> Piano Cover 🎹
import torch
import librosa
import pretty_midi
from transformers import Pop2PianoForConditionalGeneration, AutoFeatureExtractor, AutoTokenizer
from google.colab import files
from midi2audio import FluidSynth
from IPython.display import Audio, display

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Working on: {device} 🚀")

model_id = "sweetcocoa/pop2piano"
model = Pop2PianoForConditionalGeneration.from_pretrained(model_id).to(device)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

print("Model loaded successfully! ✅")

def process_audio(audio_path):
    # Load audio
    audio, sr = librosa.load(audio_path, sr=44100)
    
    # Extract rhythm using librosa significantly instead of essentia
    tempo, beat_frames = librosa.beat.beat_track(y=audio, sr=sr)
    
    # Prepare inputs
    inputs = feature_extractor(
        audio, 
        sampling_rate=sr, 
        return_tensors="pt"
    ).to(device)
    
    target_composer = "pop2piano_kpop_v1"
    print(f"Generating piano cover with style: {target_composer}...")
    
    with torch.no_grad():
        model_output = model.generate(
            input_features=inputs["input_features"],
            composer=target_composer
        )
        
    return model_output, inputs

print("\n>>> Upload your Song (MP3/WAV) below:")
uploaded = files.upload()

if uploaded:
    audio_path = next(iter(uploaded))
    print(f"Processing: {audio_path}")
    
    try:
        model_output, inputs = process_audio(audio_path)
        
        # Decode to MIDI using the processor logic (with our cheat installed)
        from transformers import Pop2PianoProcessor
        processor = Pop2PianoProcessor.from_pretrained(model_id)
        
        midi = processor.batch_decode(model_output, feature_extractor_output=inputs)
        
        midi_filename = "piano_output.mid"
        wav_filename = "piano_output.wav"
        
        # Save MIDI
        processor.save_as_midi(midi, midi_filename)
        
        # Convert MIDI to Audio for preview
        soundfont = "/usr/share/sounds/sf2/FluidR3_GM.sf2"
        FluidSynth(sound_font=soundfont).midi_to_audio(midi_filename, wav_filename)
        
        print("\n--- 🎶 Result ---")
        display(Audio(wav_filename))
        files.download(midi_filename)
        files.download(wav_filename)
        
    except Exception as e:
        print(f"Error during generation: {e}")
        import traceback
        traceback.print_exc()